# Import Library

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import time
import base64
import http.client
import json
import os
import sys, shutil
from PIL import Image, ImageDraw, ImageFont
import openpyxl

# Directory

In [ ]:
Directory = '#####'
Registered_Path = '#####'
Database_Path = '#####'
Verification_Path = '#####'

# Register New User

In [ ]:
def Register_New_User():
    Name = input('Please Enter Your Name : ')

    global File_Path
    File_Path = os.path.join(Directory, Name)

    if not os.path.exists(File_Path):
        os.makedirs(File_Path)
    
    return Name

# Capture Image From Live Video

In [ ]:
def Capture_Image():
    webcam = cv2.VideoCapture(0)
    Count = 0

    while Count < 1:
        Count = Count + 1

        # Capture frame-by-frame
        ret, frame = webcam.read()

        # Display the resulting frame
        cv2.imshow('frame', frame)

        # Capture Image in the videos
        cv2.imwrite(File_Path + '/' + str(Name) + str(Count + 666) + '.jpg', frame)

        time.sleep(0.1)

        key = cv2.waitKey(10)
        if key == 27:
            break

    webcam.release()
    cv2.destroyAllWindows()

    print('Information Stored Successfully')

# Remove Temp and Save to Registered File

In [ ]:
def Move_File():
    for All_Folders in os.listdir(Directory):
        shutil.move(Directory + All_Folders, Registered_Path)

# Face Recognition System

## Import Dependencies

In [ ]:
# Set this variable to True to print all server responses.
_print_responses = False

# Your Sighthound Cloud token. More information at
# https://www.sighthound.com/support/creating-api-token
_cloud_token = '#####'

# The cloud server to use, here we set the development server.
_cloud_host = "dev.sighthoundapi.com"

# A set in which to gather object names during step 1.
_object_ids = set()

# The name of the group to which we will add objects (step 2), train (step 3),
# and test with (step 4).
_group_name = '#####'

# The directory where annotated test images will be written.
_output_folder = "out"

## Step 1

In [ ]:
def step1_upload_images(train_path):
    """Upload all training images.

    @param  train_path  The path to the training image directory. This expects
                        images to be organized in directories by object name as
                        in "sighthound-cloud-tutorial/images/training".
    """
    print("Step 1: Uploading training images")
    # Look for directories in our training folder. The names of each directory
    # will be used as the object id for the images within.
    for name in os.listdir(train_path):
        base_path = os.path.join(train_path, name)
        if os.path.isdir(base_path):
            # Upload all image files within the directory.
            print("  Adding images for object id " + name)
            for training_file in os.listdir(base_path):
                file_path = os.path.join(base_path, training_file)
                if is_image(file_path):
                    print("    Uploading file " + training_file)
                    add_training_image(file_path, name)

                    # Track all object ids for group creation in step 2.
                    _object_ids.add(name)

    print("Step 1 complete\n")

In [ ]:
def add_training_image(image_path, object_id):
    """Submit an image to Sighthound Cloud for training.

    @param  image_path  File path to the image to analyze. The filename will be
                        used as the image id
    @param  object_id   The id of the object (person) captured by this image.
    """

    with open(image_path, "rb") as f:
        data = f.read()
        base64_image = base64.b64encode(data)
        base64_image = Encoder(base64_image)
        
        
    params = json.dumps({"image": base64_image})

    url_path = "/v1/image/%s?train=manual&objectType=person&objectId=%s" % \
            (os.path.basename(image_path), object_id)

    send_request("PUT", url_path, params)

In [ ]:
def is_image(filename):
    """A naive utility function to determine images via filename extension.

    @param  filename  The filename to examine.
    @return is_image  True if the file appears to be an image.
    """
    return filename.endswith('.png') or filename.endswith('.jpeg') or \
            filename.endswith('.jpg') or filename.endswith('.bmp')

## Step 2

In [ ]:
def step2_create_group():
    """Create a group named via _group_name with the members from step 1."""
    print ("Step 2: Creating group")
    print ("  Adding objects %s to group %s" % (str(_object_ids), _group_name))

    params = json.dumps({"objectIds": list(_object_ids)})
    send_request("PUT", "/v1/group/" + _group_name, params)
    
    print("Step 2 complete\n")


In [ ]:
def Add_Object_To_Group(Name1):
    params = json.dumps({"objectIds": [Name1]})
    send_request("POST", "/v1/group/%s" % _group_name , params)

In [ ]:
def send_request(request_method, request_path, params):
    """A utility function to send API requests to the Sighthound Cloud server.

    This function will abort the script with sys.exit(1) on API errors.
    
    @param  request_method  The request method, "PUT" or "POST".
    @param  request_path    The URL path for the API request.
    @param  params          The parameters of the API request, if any.
    @return response_body   The body of the response.
    """
    # Send the request.
    headers = {"Content-type": "application/json",
               "X-Access-Token": _cloud_token}
    conn = http.client.HTTPSConnection(_cloud_host)
    conn.request(request_method, request_path, params, headers)

    # Process the response.
    response = conn.getresponse()
    body = response.read()
    error = response.status not in [200, 204]

    if _print_responses or error:
        print(response.status, body)

    if error:
        sys.exit(1)

    return body

## Step 3

In [ ]:
def step3_train_group():
    """Train the group we created in step 2 to prepare it for recognition."""
    print("Step 3: Training group")
    print("  Sending train request for group %s" % _group_name)

    send_request("POST", "/v1/group/%s/training" % _group_name, None)
    
    print("Step 3 complete\n")


## Step 4

In [ ]:
def Encoder(Image):
    if isinstance(Image, bytes):  
        return str(Image, encoding='utf-8');  
    return json.JSONEncoder.default(self, Image)  

In [ ]:
def step4_test(test_path):
    """Send images to our newly trained group to test its recognition."""
#     print "Step 4: Beginning tests"
    # Create the output directory.
    if not os.path.exists(_output_folder):
        os.mkdir(_output_folder)

    for test_file in os.listdir(test_path):
        file_path = os.path.join(test_path, test_file)
        if not is_image(file_path):
            continue

        final_test_path = os.path.join(test_path, test_file)

        with open(final_test_path, "rb") as t:
            data = t.read()
            base64_image = base64.b64encode(data)
            base64_image = Encoder(base64_image)
            
        params = json.dumps({"image": base64_image})
        url_path = "/v1/recognition?groupId=" + _group_name
        response = json.loads(send_request("POST", url_path, params))
        

        # Annotate the image
        image = Image.open(file_path)
        font = ImageFont.load_default
        draw = ImageDraw.Draw(image)

        for face in response['objects']:
            # Retrieve and draw a bounding box for the detected face.
            json_vertices = face['faceAnnotation']['bounding']['vertices']
            vert_list = [(point['x'], point['y']) for point in json_vertices]
            draw.polygon(vert_list)

            # Retrieve and draw the id and confidence of the recongition.
            
            name = face['objectId']
            confidence = face['faceAnnotation']['recognitionConfidence']
            draw.text(vert_list[0], "%s - %s" % (name, confidence))

        image.save(os.path.join(_output_folder, test_file))
        
    
    return name, confidence

#     print "Step 4 complete\n"

# Delete Users

In [ ]:
def Delete_User(Name):
    send_request("DELETE", "/v1/object/%s" % Name , None)
    print('Delete Successfully')

# Check Object in Group

In [ ]:
def Check_User():
    Object_List = []
    Object_List = send_request("GET", "/v1/object", None)
    print (Object_List)


# Take Photo For Verification

In [ ]:
def Capture_Image_For_Verification():
    webcam = cv2.VideoCapture(0)

    # Capture frame-by-frame
    ret, frame = webcam.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Capture Image in the videos
    cv2.imwrite(Verification_Path + '1.jpg', frame)

    webcam.release()
    cv2.destroyAllWindows()

In [ ]:
def DeleteGroup():
    print(send_request("GET", "/v1/group", None))

# Main Function

In [ ]:
if __name__ == '__main__':

    root_dir = '#####'
    
    print ('1. Register')
    print ('2. Verification')
    print ('3. Delete User')
    print ('4. Check User')
    Choice = input('Please Enter Your Choice : ')
    print (' ')
    
    if int(Choice) == 1:
        Name1 = Register_New_User()
        Capture_Image()
        step1_upload_images(os.path.join(root_dir, "Temp"))
        #step2_create_group()
        Add_Object_To_Group(Name1)
        step3_train_group()
        Move_File()
        
    elif int(Choice) == 2:
        Capture_Image_For_Verification()
        Name2, Confidence = step4_test(os.path.join(root_dir, "Verification"))
        print(Name2, Confidence)
    
    elif int(Choice) == 3:
        Name_Deleted = input('Please Enter The Name To Be Deleted : ')
        Delete_User(Name_Deleted)
        
    elif int(Choice) == 4:
        Check_User()
        
    elif int(Choice) == 5:
        DeleteGroup()